In [2]:
import pandas as pd

data = pd.read_excel("data/Bsal_Standorte_06072022.MV_110722.xlsx")

data.sort_values("Jahr") #sorting by year

,Standort,Infektionstyp,Remarks,Anzahl Arten,Feuersalamander,Bergmolch,Fadenmolch,Teichmolch,Kammmolch,Gelbbauchunke,Grasfrosch,Erdkröte,Geburtshelferkröte,Grünfrösche,Jahr,X_Coord,Y_Coord
0,Vichtbach/Rott (Massensterben),Erstinfektion,retrospective from 2004,1,1,0,0,0,0,0,0,0,0,0,2004,6.185062,50.656809
1,Bunderbos (Massensterben),Erstinfektion,,1,1,0,0,0,0,0,0,0,0,0,2010,5.747381,50.913100
2,"Stolberg, Binsfeldhammer",Fraglich,dead newt found in 2011,1,0,1,0,0,0,0,0,0,0,0,2011,6.243033,50.756521
3,Robertville (Massensterben),Erstinfektion,,1,1,0,0,0,0,0,0,0,0,0,2013,6.118253,50.455664
4,Solchbachtal,Fraglich,"infected Sasa 2015, 2016, 2018",3,1,1,1,0,0,0,0,0,0,0,2015,6.270100,50.701780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Densborn 2,erstinfektion,verified by skin swab,1,1,0,0,0,0,0,0,0,0,0,2022,6.589851,50.130913
113,Oeventrop,Erstinfektion,,1,1,0,0,0,0,0,0,0,0,0,2022,8.129389,51.378028
114,Freienohl,Erstinfektion,,1,1,0,0,0,0,0,0,0,0,0,2022,8.208370,51.393784
115,Densborn 4,Erstinfektion,verified by skin swab,1,1,0,0,0,0,0,0,0,0,0,2022,6.576497,50.122584


#### Utilizing [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) (great-circle distance) to calculate shortest distance between two points:
We use this as we are going to compare year with year+1 and look at all possible distances between all locations where Bsal was detected, to see which distance was the shortest, as it is the most likely, that Bsal was transmitted from the respective locations (of year and year+1) with minimum distance to each other.

In [3]:
from math import cos, asin, sqrt, pi

def haversine(lat0, lon0, lat1, lon1):
    p = pi/180
    a = 0.5 - cos((lat1-lat0)*p)/2 + cos(lat0*p) * cos(lat1*p) * (1-cos((lon1-lon0)*p))/2
    return 12742 * asin(sqrt(a)) #12742=2*R, R=radius of our planet

## Our task is to iterate over the list, finding min distance in km between two respective locations of a year and the following year and print out the coordinates as well as respective the places and years:
### It is quite likely, that transmission happened between the respective mentioned points.
This was a much more difficult task than first expected, due to numerous data-handling (file-formatting reasons) with strange errors that took a long time to debug and the logic was also less trivial than expected, as we had to use a triple-nested for-loop. The code will work unchanged if data is appended to the original dataset (excel spreadsheet). All distances between locations of a year are compared with its next year, the shortest distances will be stored to a .csv .

#### Logic: 
Start at year0. Pick first location in year0, then measure distance with all locations of year1. Once done take next location in year0 and compare with all locations of year1 again. Store all results in temp_dist dataframe. Once all locations of year0 have been iterated over, move on to the next year, which will become year0 and repeat the process. At each year0-year1 comparison store all the results in temp_dist and once the year0-year1 comparison is over store the respective smallest distance in temp_dist to min_distances.
Outer for-loop goes from year to year, and sets our year0. Middle for-loop goes through all locations in year0. inner for-loop goes through all locations in year1 and calculates distances between two respective locations of year0 and year1. Before starting a new cycle outer for-loop then takes the minimum distance entry (row) in the temp_dist dataframe and adds it to the min_distances dataframe, which is finally gonna be saved to the .csv once the outer for-loop is finished and all years have been iterated over.

We can use this result to show the transmission paths visually on our map.

In [108]:
years = data["Jahr"].drop_duplicates() #creating a pandas series, only containing years

#creating empty dataframe, storing the min. distances between two locations along with extra info
min_distances = pd.DataFrame(columns=['Distance','Loc0','Loc1','Lat0','Lon0','Lat1','Lon1','Year0','Year1']) 

yearsL=years.tolist() #so years have a regular index (as pandas series complicates things...)


for year0 in yearsL:    
    #print('outer loop starting')
    print(year0) #current iteration, comparing year0 with next year  
    
    #temp_dist dataframe will contain all combinations of locations of the currently iterated over
    #year and its following year after the full iteration is complete, the shortest distance within
    #temp_dist is appended to min_distances
    temp_dist = pd.DataFrame(columns=['Distance','Loc0','Loc1','Lat0','Lon0','Lat1','Lon1','Year0','Year1'])
    
    #locs0 is a dataframe that only includes the current year's data
    locs0 = data[data["Jahr"] == year0] 
    
    
    #stop the iteration, if we arrived at the last year
    nexty_index = yearsL.index(year0) + 1
    if(nexty_index == len(yearsL)): 
        break
    else:
        locs1 = data[data["Jahr"] == yearsL[yearsL.index(year0)+1]]
        

    
    for i in locs0.iterrows():
        #print('middle loop starting')

        i = list(i) #converting pandas series to a list for easier handling
    
        lat0 = i[1][16]  #to access float of Latitude(Y_Coord)
        lon0 = i[1][15]  #to access float of Longitude(X_Coord)                        
        loc0 = i[1][0]   #to access location (Standort) of inner loop (year1)
        
        

        for ele in locs1.iterrows():
            #print("innest loop starting")
            
            ele = list(ele) #converting pandas series to list for easier management

            lat1 = ele[1][16]  #to access float of Latitude(Y_Coord)
            lon1 = ele[1][15]  #to access float of Longitude(X_Coord)                        
            loc1 = ele[1][0]   #to access location (Standort) of inner loop (year1)
            year1 = ele[1][14] #to access year (Jahr) of year1 (next after year0)

            #print(loc0+' this is Location0 (loc0) from following year')
            #print(loc1+' this is Location1 (loc1) from following year')

            tDis = haversine(lat0,lon0,lat1,lon1)
            #print(str(tDis)+' km')
            
            #temp_dist includes ALL comparisons between one year and the next year!
            #the following command adds a row to the temp_dist dataframe containing new info
            temp_dist.loc[len(temp_dist.index)] = [tDis,loc0,loc1,lat0,lon0,lat1,lon1
                                ,year0,year1]
            
    
    #print(temp_dist) #prints all distances between two neighboring years locations
    
    #calling nsmallest method on dataframe is like calling min() function on a list, (1,['Distance']) means
    #that it is only taking the 1 smallest element from the temp_distance dataframe (df of 1 row created):
    min_dist = temp_dist.nsmallest(1, ['Distance']) 
    
    #concatenate temp_dist with min_distances dataframe
    min_distances = pd.concat([min_distances,min_dist],axis=0)
    
#print-out of final result
print('min_distances incoming')
print(min_distances)

#export final dataframe concluding only the minimum distances between each year:
min_distances.to_csv(r'min_distances.csv', index=False, header=True)

2004
2010
2011
2013
2015
2016
2017
2018
2019
2020
2021
2022
min_distances incoming
      Distance                             Loc0  \
0    41.939265   Vichtbach/Rott (Massensterben)   
0    38.919212        Bunderbos (Massensterben)   
0    34.593279         Stolberg, Binsfeldhammer   
3    17.107302      Robertville (Massensterben)   
2     5.942631                       Weiße Wehe   
1     7.335911                       Simonskall   
50    1.253469                 Frenck/Weberbach   
74    1.016698                      Teufelspütz   
447   2.731169  Witten, Borbach (Massensterben)   
161   0.041312                        Memmingen   
17   14.347340              Tongrube Niederprüm   

                              Loc1       Lat0       Lon0       Lat1  \
0        Bunderbos (Massensterben)  50.656809   6.185062  50.913100   
0         Stolberg, Binsfeldhammer  50.913100   5.747381  50.756521   
0      Robertville (Massensterben)  50.756521   6.243033  50.455664   
3    Belgenbachtal (